In [35]:
import base64
from os import listdir
from os.path import isfile, join
from typing import Dict, List, Literal, Optional, TypedDict, Union

from IPython.display import HTML, Markdown
from langchain.agents import AgentType, initialize_agent
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.agent_toolkits import GmailToolkit
from langchain_community.tools.gmail.utils import (build_resource_service,
                                                   get_gmail_credentials)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from rich.pretty import pprint


def pretty_print(title: str = None, content: str = None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

In [36]:
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="tmp/credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)
tools = toolkit.get_tools()

tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x128486ed0>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x128486ed0>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x128486ed0>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x128486ed0>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x128486ed0>)]

In [37]:
model = ChatOpenAI(
    model="gpt-4-vision-preview", temperature=0, max_tokens=1024 * 2
)

#from langchain_community.llms.ollama import Ollama
# llm = Ollama(
#     base_url="http://localhost:11434",
#     model="gemma:2b",
#     temperature=0, 
# )

agent = initialize_agent(tools=tools, llm=model, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent

AgentExecutor(verbose=True, tags=['structured-chat-zero-shot-react-description'], agent=StructuredChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\ncreate_gmail_draft: Use this tool to create a draft email with the provided message fields., args: {{\'message\': {{\'title\': \'Message\', \'description\': \'The message to include in the draft.\', \'type\': \'string\'}}, \'to\': {{\'title\': \'To\', \'description\': \'The list of recipients.\', \'type\': \'array\', \'items\': {{\'type\': \'string\'}}}}, \'subject\': {{\'title\': \'Subject\', \'description\': \'The subject of the message.\', \'type\': \'string\'}}, \'cc\': {{\'title\': \'Cc\', \'description\': \'The list of CC recipients.\', \'type\': \'array\', \'items\': {{\'type\': \'string\'}}

In [38]:
agent.invoke("""Send an email 

             To: chris.at.de@gmail.com

             Subject: Hier ist was neues, bitte aufpassen 
             
             Message: Hallo, Xinyue, es ist eine gute Nachrichten, wir müssen morgen nicht zu früh aufstehen.
""")



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "send_gmail_message",
  "action_input": {
    "to": "chris.at.de@gmail.com",
    "subject": "Hier ist was neues, bitte aufpassen",
    "message": "Hallo, Xinyue, es ist eine gute Nachrichten, wir müssen morgen nicht zu früh aufstehen."
  }
}
```
Observation: Message sent. Message Id: 18dec4f946bf05eb
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The email has been successfully sent to chris.at.de@gmail.com with the subject 'Hier ist was neues, bitte aufpassen' and the message 'Hallo, Xinyue, es ist eine gute Nachrichten, wir müssen morgen nicht zu früh aufstehen.'"
}
```

> Finished chain.


{'input': 'Send an email \n\n             To: chris.at.de@gmail.com\n\n             Subject: Hier ist was neues, bitte aufpassen \n             \n             Message: Hallo, Xinyue, es ist eine gute Nachrichten, wir müssen morgen nicht zu früh aufstehen.\n',
 'output': "The email has been successfully sent to chris.at.de@gmail.com with the subject 'Hier ist was neues, bitte aufpassen' and the message 'Hallo, Xinyue, es ist eine gute Nachrichten, wir müssen morgen nicht zu früh aufstehen.'"}

# Use LangGraph to handle in batch

In [39]:
structured_output_parser = StructuredOutputParser.from_response_schemas(
            [
                ResponseSchema(
                    name="subject",
                    description="Subject of the email",
                    type="string",
                ),
                ResponseSchema(
                    name="message",
                    description="Subject of the email",
                    type="string",
                ),
            ]
        )

In [40]:
class GraphState(TypedDict):
    mailto: Optional[str] = None
    subject: Optional[str] = None
    recepit_stack: Optional[List[str]] = None # A image file path list of the receipts
    current_recepit_pos:Optional[int]=None # The position of the current receipt to handle
    receipt_description: Optional[str] = None # The description of the read receipt
    results: Optional[Dict[str, Dict[str, str]]] = None # The results of the operations, receipt path and Dict[str, str]: the content of the mail: subject and message.

In [41]:
def pop_recepit_stack(state: StateGraph) -> Dict[str, str]:
    current_recepit_pos = state["current_recepit_pos"]
    if current_recepit_pos is None:
        current_recepit_pos = -1
    return {"current_recepit_pos": current_recepit_pos + 1}


def read_receipt(state: StateGraph) -> Dict[str, str]:
    recepit_stack = state["recepit_stack"]
    current_recepit_pos = state["current_recepit_pos"]
    image_path = recepit_stack[current_recepit_pos]

    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")
        prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(
                    content="""As a useful assistant you provide the user with the receipt data information as an email message related to what they have consumed, bought, and paid for. 
                    Include the product name, price, and quantity if provided. 

                    Avoid any personal information, locations, addresses (even store's), sensitive data, and numbers in the email message.
                    
                    ONLY format the response in a reasonable TABLE in HTML format (Email compatible HTML), no other paragraphs, phgrases or sentences are allowed.
                   
                    In the table, the first row, in bold font, is the summe of the total payment, afterwards the product name, price, and quantity if provided.
                    
                    Above the table, give the store or the brand name.

                    The subject of the mail should be:
                    "Receipt from [store or brand name], payment date: [date]"
                    When the date is unknown, then show "unknown" instead of the date.

                    The final result should be json:

                    subject: The email subject
                    message: The email message
                    
                    """,
                ),
                HumanMessage(
                    content=[
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                            },
                        },
                    ]
                ),
            ],
        ).partial(
            format_instructions=structured_output_parser.get_format_instructions()
        )

        res = (prompt | model | StrOutputParser()).invoke(
            {"base64_image": base64_image}
        )

        structured_res = structured_output_parser.parse(res)

        description = structured_res["message"]
        subject = structured_res["subject"]

        display(HTML(subject))
        display(HTML(description))
        return {"receipt_description": description, "subject": subject}


def send_email(state: StateGraph) -> Dict[str, str]:
    receipt_description = state["receipt_description"]
    mailto = state["mailto"]
    subject = state["subject"]
    pretty_print("mailto", mailto)
    agent.invoke(
        f"""Send an email

                 To: {mailto}

                 Subject: {subject}

                 Message: {receipt_description}
    """
    )

    recepit_stack = state["recepit_stack"]
    current_recepit_pos = state["current_recepit_pos"]
    image_path = recepit_stack[current_recepit_pos]

    if state["results"] is None:
        state["results"] = {}

    state["results"][image_path] = {
        "subject": subject,
        "receipt_description": receipt_description,
    }
    return {"results": state["results"], "receipt_description": None}


def continue_next(state: StateGraph) -> Literal["to_read_receipt", "to_finish"]:
    recepit_stack = state["recepit_stack"]
    length = len(recepit_stack)
    current_recepit_pos = state["current_recepit_pos"]
    if current_recepit_pos < length:
        return "to_read_receipt"
    else:
        return "to_finish"


workflow = StateGraph(GraphState)

workflow.add_node("pop_recepit_stack", pop_recepit_stack)
workflow.add_node("read_receipt", read_receipt)
workflow.add_node("send_email", send_email)


workflow.set_entry_point("pop_recepit_stack")

workflow.add_edge("read_receipt", "send_email")
workflow.add_edge("send_email", "pop_recepit_stack")


workflow.add_conditional_edges(
    "pop_recepit_stack",  # start node name
    continue_next,  # decision of what to do next AFTER start-node, the input is the output of the start-node
    {  # keys: return of continue_next, values: next node to continue
        "to_read_receipt": "read_receipt",
        "to_finish": END,
    },
)

app = workflow.compile()

In [42]:
image_dir = "assets/images/receipt"
image_files = [f for f in listdir(image_dir) if isfile(join(image_dir, f))]
image_file_paths = [join(image_dir, f) for f in image_files]

result = app.invoke(
    {
        "recepit_stack": image_file_paths,
        "mailto": "chris.at.de@gmail.com",
    },
    {"recursion_limit": 100}
)
pretty_print("Result:", result)

Total,"EUR 24,07"


mailto


'chris.at.de@gmail.com'



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "send_gmail_message",
  "action_input": {
    "to": "chris.at.de@gmail.com",
    "subject": "Receipt from Burger King, payment date: 25.02.24",
    "message": "<table style='width:100%;'><tr><td><strong>Total</strong></td><td><strong>EUR 24,07</strong></td></tr></table>"
  }
}
```
Observation: Message sent. Message Id: 18dec4fd794ff6a3
Thought:{
  "action": "Final Answer",
  "action_input": "The email has been successfully sent to chris.at.de@gmail.com with the subject 'Receipt from Burger King, payment date: 25.02.24'. The content of the message was delivered in HTML format displaying a table with the total amount of EUR 24,07."
}

> Finished chain.


mailto


...

Total,€16.99
Ch n Sh 18er HW,€16.99
18 Hot Wings Uptrade Bucket,€15.88
VAT (7%),€1.11


mailto


'chris.at.de@gmail.com'



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "send_gmail_message",
  "action_input": {
    "to": "chris.at.de@gmail.com",
    "subject": "Receipt from KFC, payment date: 24.02.2024",
    "message": "<table style='width:100%;'><tr><td><strong>Total</strong></td><td><strong>€16.99</strong></td></tr><tr><td>Ch n Sh 18er HW</td><td>€16.99</td></tr><tr><td>18 Hot Wings Uptrade Bucket</td><td>€15.88</td></tr><tr><td>VAT (7%)</td><td>€1.11</td></tr></table>"
  }
}
```
Observation: Message sent. Message Id: 18dec51db97926ec
Thought:```
{
  "action": "Final Answer",
  "action_input": "The email has been sent successfully to chris.at.de@gmail.com with the subject 'Receipt from KFC, payment date: 24.02.2024'."
}
```

> Finished chain.
Result:


{
│   'mailto': 'chris.at.de@gmail.com',
│   'subject': 'Receipt from KFC, payment date: 24.02.2024',
│   'recepit_stack': [
│   │   'assets/images/receipt/WechatIMG144.jpg',
│   │   'assets/images/receipt/WechatIMG145.jpg',
│   │   'assets/images/receipt/WechatIMG147.jpg',
│   │   'assets/images/receipt/WechatIMG142.jpg',
│   │   'assets/images/receipt/WechatIMG143.jpg'
│   ],
│   'current_recepit_pos': 5,
│   'receipt_description': None,
│   'results': {
│   │   'assets/images/receipt/WechatIMG144.jpg': {
│   │   │   'subject': 'Receipt from Burger King, payment date: 25.02.24',
│   │   │   'receipt_description': "<table style='width:100%;'><tr><td><strong>Total</strong></td><td><strong>EUR 24,07</strong></td></tr></table>"
│   │   },
│   │   'assets/images/receipt/WechatIMG145.jpg': {
│   │   │   'subject': 'Receipt from TotalEnergies, payment date: 24.02.2024',
│   │   │   'receipt_description': "<table style='width:100%;'><tr><td><strong>Total</strong></td><td><strong>68,47 €</strong></td></tr><tr><td>Super</td><td>1,879 €/Liter</td><td>36,44 Liter</td></tr><tr><td>VAT (19% MwSt. A)</td><td>10,93 €</td></tr><tr><td>Net Sum</td><td>57,54 €</td></tr></table>"
│   │   },
│   │   'assets/images/receipt/WechatIMG147.jpg': {
│   │   │   'subject': 'Receipt from Kaufland, payment date: 26.02.24',
│   │   │   'receipt_description': "<table style='width:100%;'><tr><td><strong>Total</strong></td><td><strong>33,45 EUR</strong></td></tr><tr><td>Banane Chiquita</td><td>1,68</td></tr><tr><td>E. Kuklaputus Ment.</td><td>0,69</td></tr><tr><td>Broccoli 500g</td><td>0,99</td></tr><tr><td>Lauchzwiebeln</td><td>0,49</td></tr><tr><td>Lachsfilets</td><td>11,98</td></tr><tr><td>Mü. Milchreis</td><td>2,67</td></tr><tr><td>Knoblauch</td><td>1,99</td></tr><tr><td>Eier Bodenhaltung</td><td>1,99</td></tr><tr><td>Hohes C Vitamin D</td><td>2,99</td></tr><tr><td>Beerenmix 150g</td><td>2,79</td></tr><tr><td>Kerrygold extra</td><td>2,99</td></tr><tr><td>Mini-Pak Choi 300g</td><td>2,58</td></tr><tr><td>Tomate, Rispen</td><td>0,71</td></tr></table>"
│   │   },
│   │   'assets/images/receipt/WechatIMG142.jpg': {
│   │   │   'subject': 'Receipt from Kaufland, payment date: unknown',
│   │   │   'receipt_description': "<table style='width:100%;'><tr><td><strong>Total</strong></td><td><strong>8,66 EUR</strong></td></tr><tr><td>Leergut Getränke</td><td>-1,90 EUR</td></tr><tr><td>Champ. braun (400g)</td><td>0,80 EUR</td></tr><tr><td>S-Hackfleisch</td><td>2,70 EUR</td></tr><tr><td>Edamame</td><td>1,79 EUR</td></tr><tr><td>KPur P.Geschnet.</td><td>3,99 EUR</td></tr><tr><td>Chinakohl</td><td>1,28 EUR</td></tr></table>"
│   │   },
│   │   'assets/images/receipt/WechatIMG143.jpg': {
│   │   │   'subject': 'Receipt from KFC, payment date: 24.02.2024',
│   │   │   'receipt_description': "<table style='width:100%;'><tr><td><strong>Total</strong></td><td><strong>€16.99</strong></td></tr><tr><td>Ch n Sh 18er HW</td><td>€16.99</td></tr><tr><td>18 Hot Wings Uptrade Bucket</td><td>€15.88</td></tr><tr><td>VAT (7%)</td><td>€1.11</td></tr></table>"
│   │   }
│   }
}